In [572]:
#STREAMS2TABLE
#written by Zachary J.J. Roper

#A script for converting raw event codes and timestamps into a trial-item dataset

In [573]:
#This notebook requires the following modules
import os
import numpy
import csv

In [574]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [575]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [576]:
events.count('2756')

6

In [577]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [578]:
#uberlist

In [579]:
'1507' in uberlist[0][2]

True

In [580]:
RT = [None]*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [581]:
#RT

In [582]:
#The following provides indexed lists for each of the individual event codes of interest 

In [583]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [584]:
#float(times[74])-float(times[70])

In [585]:
#Saccade

In [586]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [587]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [588]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [589]:
#print(TimeTrials[0])

In [590]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [591]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0

In [592]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [593]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [594]:
#RT

In [595]:
checksum = sum(error)+sum(abort)+sum(catch)

In [596]:
checksum

470

In [597]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0
2 0 1 0 1 0 203.98070000000007
3 0 0 0 1 0 179.97810000000027
4 1 0 0 0 0 0
5 0 1 0 1 0 187.96540000000095
6 1 0 0 0 0 0
7 0 1 0 1 0 189.97239999999874
8 1 0 0 0 0 0
9 0 1 0 1 0 167.97709999999643
10 0 1 0 1 0 193.94540000000416
11 1 0 0 0 0 0
12 0 1 0 1 0 188.0063999999984
13 0 1 0 1 0 181.98540000000503
14 1 0 0 0 0 0
15 1 0 0 0 0 0
16 1 0 0 0 0 0
17 1 0 0 0 0 0
18 0 1 0 1 0 95.96919999999955
19 0 1 0 1 0 200.00759999999718
20 1 0 0 0 0 0
21 1 0 0 0 0 0
22 1 0 0 0 0 0
23 0 1 0 1 0 219.99589999999444
24 0 0 0 1 0 185.9994000000006
25 0 0 0 1 0 183.99240000000282
26 1 0 0 0 0 0
27 0 1 0 1 0 191.97939999999653
28 0 1 0 1 0 164.00389999999607
29 0 1 0 1 0 165.9701000000059
30 0 1 0 1 0 169.98409999998694
31 0 1 0 1 0 199.9667000000045
32 0 0 0 1 0 171.99109999999928
33 0 1 0 1 0 204.0216999999975
34 0 1 0 1 0 183.99239999998827
35 0 1 0 1 0 127.95909999999276
36 0 1 0 1 0 179.9783999999927
37 0 1 0 1 0 183.99210000000312
38 0 1 0 1 0 207.99469999999565
39 1 0 0 0 0 0
40 0 1

594 0 0 0 1 0 147.98850000009406
595 1 0 0 0 0 0
596 0 1 0 1 0 198.000400000019
597 0 1 0 1 0 179.97809999994934
598 0 1 0 1 0 191.97959999996237
599 0 1 0 1 0 201.97360000014305
600 0 0 0 1 0 198.00069999997504
601 0 1 0 1 0 189.97240000008605
602 1 0 0 0 0 0
603 0 1 0 1 0 163.96280000009574
604 0 1 0 1 0 207.9949000000488
605 1 0 0 0 0 0
606 0 1 0 1 0 197.9597000000067
607 1 0 0 0 0 0
608 0 1 0 1 0 200.00769999995828
609 0 1 0 1 0 179.9784000001382
610 1 0 0 0 0 0
611 1 0 0 0 0 0
612 1 0 0 0 0 0
613 0 1 0 1 0 191.97959999996237
614 0 1 0 1 0 168.0179000000935
615 0 1 0 1 0 179.9782000000123
616 1 0 0 0 0 0
617 0 1 0 1 0 193.98670000000857
618 1 0 0 0 0 0
619 1 0 0 0 0 0
620 1 0 0 0 0 0
621 0 1 0 1 0 157.98259999998845
622 1 0 0 0 0 0
623 0 1 0 1 0 231.99729999992996
624 1 0 0 0 0 0
625 0 1 0 1 0 213.97519999998622
626 0 1 0 1 0 337.9609000000637
627 0 1 0 1 0 190.01359999994747
628 1 0 0 0 0 0
629 0 1 0 1 0 183.95109999994747
630 0 1 0 1 0 219.99619999993593
631 1 0 0 0 0 0
632 1 0 0

974 1 0 0 0 0 0
975 1 0 0 0 0 0
976 0 0 0 1 0 187.96539999987
977 0 1 0 1 0 198.08250000001863
978 0 0 0 0 1 0
979 1 0 0 0 0 0
980 0 1 0 1 0 191.97970000002533
981 0 0 0 1 0 188.00640000007115
982 0 1 0 1 0 193.9865999999456
983 0 1 0 1 0 219.9551999999676
984 0 1 0 1 0 227.9832999999635
985 0 1 0 1 0 169.9841000000015
986 0 1 0 1 0 212.00900000007823
987 1 0 0 0 0 0
988 1 0 0 0 0 0
989 0 1 0 1 0 188.00639999983832
990 0 1 0 1 0 173.9981000002008
991 0 1 0 1 0 149.99560000002384
992 1 0 0 0 0 0
993 0 1 0 1 0 213.9749999998603
994 0 1 0 1 0 221.96219999995083
995 0 1 0 1 0 159.98979999986477
996 1 0 0 0 0 0
997 1 0 0 0 0 0
998 1 0 0 0 0 0
999 0 1 0 1 0 202.01470000017434
1000 1 0 0 0 0 0
1001 0 1 0 1 0 185.9993999998551
1002 1 0 0 0 0 0
1003 0 1 0 1 0 211.9680000001099
1004 1 0 0 0 0 0
1005 1 0 0 0 0 0
1006 0 1 0 1 0 170.02490000007674
1007 1 0 0 0 0 0
1008 1 0 0 0 0 0
1009 1 0 0 0 0 0
1010 1 0 0 0 0 0
1011 0 0 0 1 0 236.01150000002235
1012 1 0 0 0 0 0
1013 1 0 0 0 0 0
1014 1 0 0 0 0 0


In [598]:
sum(catch)

6